# Generalize SSI to N neurons

## Setup 

setup fisher_info_limits

In [2]:
# import packages
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pickle5 as pickle
from scipy.stats import spearmanr, wilcoxon
from matplotlib.gridspec import GridSpec as GS
import math
from scipy.stats import poisson
from scipy.optimize import minimize_scalar
from scipy.special import radian as rad
from scipy.stats import circstd
from tqdm import tqdm
from scipy.special import gammaln
from scipy.optimize import curve_fit

# setup project path
main_dir = '/Users/steeve_laquitaine/Desktop/FISHER-paper/code/fisher-info-limits/'
os.chdir(main_dir)

# import custom package
import ssiProjectModule as ssiMod

# setup parameters
sample_size = 500
theta_step = 2
stim = np.arange(0, 360, theta_step) # stimulus direction space

# setup Bayesian parameters
N_TRIALS = 500

# setup paths
dat_dir_in = main_dir + 'data/decoding_analysis/'
data_dir = 'data/bayer_decoding_error/'+f'Bayes_Dec_Errs_{sample_size}reps_thetastep{theta_step}'+'.npz'
title = f'Stretch_BioQuads_multiplied_w_mean_g__theta_step={theta_step}_original_g_20240509_AREA.pkl'

constant = 360/np.sqrt(2*np.pi*math.e)

with open(dat_dir_in + title, 'rb') as rick:
    data_dict = pickle.load(rick)

bio_idx = np.where(data_dict['all_multipliers']==1.)[0][0]

chosen_quads = np.array([201, 125, 104, 491])

## Functions

In [ ]:
def create_tuning_curves(n_neurons: int, amplitude: float, width: float, baseline:float):
    """create tuning curves
    """
    # setup the preferred angle of each neuron
    preferred_angles = np.linspace(0, 360, n_neurons, endpoint=False) 

    # create tuning curves
    tuning_c = []
    for ix, pref_angle  in enumerate(preferred_angles):
        tuning_c.append(ssiMod.VON_MISES(th=stim, a=amplitude, th_0=pref_angle, s=width, baseline=baseline))
    return np.array(tuning_c)



def test_ssi(tuning_curves: np.array):
    """test new ssi vs original ssi computation 
    for 1 to 4 neurons"""

    # compute original ssi
    ssi = ssiMod.MC_SSI(tuning_curves)   

    # compute new ssi with 2 neurons
    ssi2 = ssiMod.MC_SSI2(tuning_curves)

    # unit-test
    assert ssi.shape == (len(stim),), "ssi should be same length as stimulus count"
    assert np.allclose(ssi, ssi2,  atol=1e-08), "ssi should be the same"

## Unit-tests

In [37]:
%%time 

# n_neurons for a tiling with 20 neurons (Kriegeskorte, Nature, Rev, Neuro)
# amplitude for 19 spikes/secs (Albright, 1984)  (from Gaussian fit)
# width for 32 deg std (~75 deg FWHM), validated across species (from Gaussian fit)
tc_prms = {"amplitude": 780, "width": 0.29, "baseline": 0}

# unit-test 2 neurons
tuning_curves = create_tuning_curves(n_neurons=2, **tc_prms)
test_ssi(tuning_curves) 

# unit-test 3 neurons
tuning_curves = create_tuning_curves(n_neurons=3, **tc_prms)
test_ssi(tuning_curves) # unit-test

# unit-test 4 neurons
tuning_curves = create_tuning_curves(n_neurons=4, **tc_prms)
test_ssi(tuning_curves) # unit-test